In [1]:
import numpy as np
import pandas as pd
import os
from os import path
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix

import pickle

In [2]:
import tensorflow as tf
import re

import keras
from sklearn.model_selection import train_test_split

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, CuDNNLSTM
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [4]:
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
tweetDF = pd.read_csv("premade_datasets/training_dataset_non_lemmatized.csv")

In [6]:
tweetDF.head()

,full_text,prep_text,sentiment
0,Now I can’t I see it😭😭 https://t.co/8ZPKtJ2SLd,now i i see it URL,0
1,No this is a REAL FEAR 😭😭😭 https://t.co/68HkUu...,no this is a real fear URL,0
2,@kwadwosheldon Grown up paa ow 😓,USERNAME grown up paa ow,0
3,God I hate being responsible sometimes. My fam...,god i hate being responsible sometimes my fami...,0
4,@LUXMINAJJ @BenjaminEnfield peaches Like it 😭,USERNAME USERNAME peaches like it,0


In [7]:
X_input= tweetDF['prep_text']
y_input= tweetDF['sentiment']

In [8]:
def postProcessing(tweet):
    text = tweet
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    
    return ' '.join(words)

In [9]:
X_input.apply(postProcessing)

0                                                    see URL
1                                              real fear URL
2                                      USERNAME grown paa ow
3          god hate responsible sometimes family going un...
4                             USERNAME USERNAME peaches like
                                 ...                        
2169656                                             new gens
2169657                                   USERNAME fuck yeah
2169658    USERNAME bts comeback 10 de junho btsbarmy com...
2169659    mental health awareness month join us may cont...
2169660                                             USERNAME
Name: prep_text, Length: 2169661, dtype: object

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_input, y_input, test_size=0.20, random_state=42, stratify=y_input)

In [13]:
from LSTMEmbedding import LSTMEmbedding 

In [14]:
le = LSTMEmbedding()

In [15]:
le.initialize_tokenizer(8000, 30)
le.fit_tokenizer(X_train)

In [16]:
le.read_glove_vector('glove.twitter.27B.200d.txt')

In [17]:
emb_layer = le.create_embedding_layer()

In [18]:
X_train_indices = le.text_to_sequences(X_train)
X_val_indices = le.text_to_sequences(X_val)

In [19]:
lr = .001
hidden_nodes = [150, 100,  50] 
max_epochs = 100
patience = 5
batch_size = 128
dropout_rate = 0.6
bidirectional = True
shape =X_train_indices.shape[1]


In [20]:
def compile_model(model, lr, loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam'):

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) if optimizer=='adam' else tf.keras.optimizers.SGD(learning_rate=lr)
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    return model

In [21]:
model = le.build_model(hidden_nodes, input_shape=shape, embedding_layer=emb_layer,bidirectional=True, dropout_rate=dropout_rate)
model = compile_model(model, lr)

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 200)           61096200  
                                                                 
 bidirectional (Bidirectiona  (None, 30, 300)          422400    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 30, 300)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 30, 200)          321600    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 30, 200)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 100)              1

In [44]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)                    
history = model.fit(X_train_indices, y_train, epochs=max_epochs, batch_size=batch_size, validation_data=(X_val_indices, y_val),
                            shuffle=False, callbacks=[callback])

Epoch 1/100
13561/13561 [==============================] - 450s 33ms/step - loss: 0.5607 - accuracy: 0.7029 - val_loss: 0.5464 - val_accuracy: 0.7147
Epoch 2/100
13561/13561 [==============================] - 444s 33ms/step - loss: 0.5381 - accuracy: 0.7201 - val_loss: 0.5394 - val_accuracy: 0.7197
Epoch 3/100
13561/13561 [==============================] - 445s 33ms/step - loss: 0.5261 - accuracy: 0.7286 - val_loss: 0.5380 - val_accuracy: 0.7212
Epoch 4/100
13561/13561 [==============================] - 445s 33ms/step - loss: 0.5156 - accuracy: 0.7360 - val_loss: 0.5401 - val_accuracy: 0.7205
Epoch 5/100
13561/13561 [==============================] - 445s 33ms/step - loss: 0.5058 - accuracy: 0.7428 - val_loss: 0.5432 - val_accuracy: 0.7196
Epoch 6/100
13561/13561 [==============================] - 445s 33ms/step - loss: 0.4968 - accuracy: 0.7488 - val_loss: 0.5479 - val_accuracy: 0.7184
Epoch 7/100
13561/13561 [==============================] - 439s 32ms/step - loss: 0.4889 - accuracy:

In [45]:
EvaltweetDF = pd.read_csv("premade_datasets/evaluation_dataset_non_lemmatized.csv")

In [46]:
X_test= EvaltweetDF['prep_text']
y_test= EvaltweetDF['sentiment']

In [47]:
X_test.apply(postProcessing)

0                                       USERNAME aww bad
1                               bro tv break crunch time
2              USERNAME bestie cry everyday like EMO URL
3                             talking stop liking period
4      USERNAME yes ending shocking devastating cared...
                             ...                        
194    USERNAME USERNAME thanks sabrina appreciate tw...
195    USERNAME kai handle anything push everything g...
196    always pleasure meeting legend virat bhai stil...
197    long breathing manchester united always get fu...
198                      USERNAME thank much support EMO
Name: prep_text, Length: 199, dtype: object

In [48]:
X_test_indices = le.text_to_sequences(X_test)


In [49]:
y_pred = model.predict(X_test_indices)
y_pred = np.around(y_pred)

In [50]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print("\n")
print('Classification Report')
print(classification_report(y_test, y_pred))

Confusion Matrix
[[90  9]
 [13 87]]


Classification Report
              precision    recall  f1-score   support

           0       0.87      0.91      0.89        99
           1       0.91      0.87      0.89       100

    accuracy                           0.89       199
   macro avg       0.89      0.89      0.89       199
weighted avg       0.89      0.89      0.89       199



In [51]:
dfs = [pd.read_csv(path.join('reddit/final',x)) for x in os.listdir("reddit/final") if path.isfile(path.join("reddit/final",x))]
reddit = pd.concat(dfs)

In [52]:
X_test= reddit['prep_text']
y_test= reddit['sentiment']

In [53]:
X_test.apply(postProcessing)

0      ukraine collects russian dead war rages multip...
1      lawsuit claims bastrop county deputies torture...
2      israeli police beat mourners murdered al jazee...
3      us lawmakers decry israeli forces attacking fu...
4      amelia baca baca family attorneys demand murde...
                             ...                        
104    underground facility hold soviet warship perfe...
105    nostalgic survivors know love beautiful images...
106    happy international womens wish women h w new ...
107    one largest abandoned dive sites canada radioa...
108    hailed heroes scottish gardeners rescued trio ...
Name: prep_text, Length: 218, dtype: object

In [54]:
X_test_indices = le.text_to_sequences(X_test)


In [55]:
y_pred = model.predict(X_test_indices)
y_pred = np.around(y_pred)

In [56]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print("\n")
print('Classification Report')
print(classification_report(y_test, y_pred))

Confusion Matrix
[[97 12]
 [20 89]]


Classification Report
              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86       109
         1.0       0.88      0.82      0.85       109

    accuracy                           0.85       218
   macro avg       0.86      0.85      0.85       218
weighted avg       0.86      0.85      0.85       218



In [38]:
EvalPapertweetDF = pd.read_csv("premade_datasets/evaluation_paper.csv")

In [39]:
X_test= EvalPapertweetDF['prep_text']
y_test= EvalPapertweetDF['sentiment']

In [40]:
X_test.apply(postProcessing)

0      USERNAME loovvee kindle2 dx cool 2 fantastic r...
1              reading kindle2 love lee childs good read
2               ok first assesment kindle2 fucking rocks
3      USERNAME youll love kindle2 ive mine months ne...
4             USERNAME fair enough kindle2 think perfect
                             ...                        
354    using latex lot typeset mathematics looks hideous
355    note hate word hate pages hate latex said hate...
356         ahh back real text editing environment latex
357    trouble iran see hmm iran iran far away flocko...
358    reading tweets coming iran whole thing terrify...
Name: prep_text, Length: 359, dtype: object

In [32]:
X_test_indices = le.text_to_sequences(X_test)

In [43]:
y_pred = model.predict(X_test_indices)
y_pred = np.around(y_pred)

In [44]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print("\n")
print('Classification Report')
print(classification_report(y_test, y_pred))

Confusion Matrix
[[123  54]
 [ 40 142]]


Classification Report
              precision    recall  f1-score   support

           0       0.75      0.69      0.72       177
           1       0.72      0.78      0.75       182

    accuracy                           0.74       359
   macro avg       0.74      0.74      0.74       359
weighted avg       0.74      0.74      0.74       359



In [63]:
weights = model.weights

with open("Bilstm_weights.pkl", "wb") as f:
    pickle.dump(weights, f)
    